In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import os
import time
import pandas as pd

In [28]:
local = os.getcwd()
local_planilha = local + r'\Processos.xlsx'

df = pd.read_excel(local_planilha)


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  4 non-null      int64 
 1   Unnamed: 0    4 non-null      int64 
 2   Nome          4 non-null      object
 3   Advogado      4 non-null      object
 4   Processo      4 non-null      object
 5   Cidade        4 non-null      object
 6   Status        4 non-null      object
dtypes: int64(2), object(5)
memory usage: 352.0+ bytes


In [30]:
navegador = webdriver.Chrome()

In [ ]:
site = local + r'\index.html'
navegador.get(site)

In [ ]:


# posicionando o Mouse para o botão de escolher estado
for processo in df.index:

   cidade = str(df.loc[processo, 'Cidade'])
   ActionChains(navegador).move_to_element(navegador.find_element(By.XPATH, '/html/body/div/div/button')).perform()


   navegador.find_element(By.PARTIAL_LINK_TEXT, f'{cidade}').click()



   AbaHome = navegador.window_handles[0]
   AbaVerificacao = navegador.window_handles[1]

   # Indo para a nova aba aberta
   navegador.switch_to.window(AbaVerificacao)

   navegador.find_element(By.ID, 'nome').send_keys(df.loc[processo, 'Nome'])
   navegador.find_element(By.ID, 'advogado').send_keys(df.loc[processo, 'Advogado'])
   navegador.find_element(By.ID, 'numero').send_keys(df.loc[processo, 'Processo'])

   # clicando em pesquisar
   navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

   # aceitando o alerta de pesquisar no banco de dados
   navegador.switch_to.alert.accept()
   time.sleep(1)

   # pegando a informação do novo alerta, sobre ter encontrado ou não o processo

   while True:
      try:
         informacao = navegador.switch_to.alert.text
         break
      except:
         time.sleep(0.5)


   time.sleep(1)
   if informacao == 'Processo encontrado com sucesso! Deseja acessar?':
      navegador.switch_to.alert.accept()
      df.loc[processo, 'Status'] = 'Encontrado'
   else:
      df.loc[processo, 'Status'] = 'Sem resultados'
      navegador.switch_to.alert.accept()

   navegador.close()
   navegador.switch_to.window(AbaHome)

navegador.quit()
df.to_excel('Processos.xlsx')
